# Problem Formation

Given a Pattern String as an input, we want to know if it contains dark pattern in it. We use a balanced dataset cotaining all the instances in the Princeton dataset which are all dark patterns, and the instances in the 'normie.csv' file which are labeled as NOT dark patterns. Hence we have a balanced dataset consisting of pattern strings with dark pattern and without park patterns.

Then we use this labeled dataset to build and train supervised machine learning models, and select most suitable ones for our project.

----


In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary.
from sklearn.feature_extraction.text import TfidfVectorizer

# Bernoulli Naive Bayes (Similar as  MultinomialNB), this classifier is suitable for discrete data. The difference between MultinomialNB and BernoulliNB is that while  MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolen features, which means in the case of text classification, word occurrence vectores (rather than word count vectors) may be more suitable to be used to train and use this classifier.
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# Evaluation metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib

import matplotlib.pyplot as plt
# import seaborn as sns

## Data Exploration

---
Import the merged dataset, and explore the dataset.

In [2]:
data = pd.read_csv('enriched_confirm.csv')

In [3]:
data.head(5)

,Pattern String,classification
0,Ends in 07:42:09,Not_Dark
1,Ends in 07:37:10,Not_Dark
2,Ends in 02:27:10,Not_Dark
3,Ends in 04:17:10,Not_Dark
4,Ends in 01:57:10,Not_Dark


---
`check the dataset information`

There are 7952 NOT NULL instances of pattern strings in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8187 entries, 0 to 8186
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  8187 non-null   object
 1   classification  8187 non-null   object
dtypes: object(2)
memory usage: 128.0+ KB


In [5]:
# check the distribution of the target value --- classification.

print('Distribution of the tags:\n{}'.format(data['classification'].value_counts()))

Distribution of the tags:
Not_Dark    7994
Dark         193
Name: classification, dtype: int64


In [6]:
# For later training the model, we should remove the duplicate input to reduce overfitting.

data = data.drop_duplicates(subset="Pattern String")

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))


Distribution of the tags:
Not_Dark    7994
Dark         193
Name: classification, dtype: int64


---
## Data Preparation

In [7]:
# split the dataset into train and test dataset as a ratio of 80%/20% (train/test).

string_train, string_test, dark_train, dark_test = train_test_split(
    data['Pattern String'], data["classification"], train_size = .8)

---
`Encode the target vales into integers` --- 'classification'

In [8]:
encoder = LabelEncoder()
encoder.fit(dark_train)
y_train = encoder.transform(dark_train)
y_test = encoder.transform(dark_test)

In [9]:
# check the mapping of encoding results (from 0 to 1 representing 'Dark', 'Not Dark')

integer_mapping = {label: encoding for encoding, label in enumerate(encoder.classes_)}
print(integer_mapping)

{'Dark': 0, 'Not_Dark': 1}


In [10]:
# Check the frequency distribution of the training pattern classification with pattern classification names.

(unique, counts) = np.unique(dark_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[['Dark' 154]
 ['Not_Dark' 6395]]


In [11]:
# Check the frequency distribution of the encoded training pattern classification with encoded integers.

(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  154]
 [   1 6395]]


In [12]:
# Check the frequency distribution of the encoded testing pattern classification with encoded integers.

(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0   39]
 [   1 1599]]


---
`Encode the textual features into series of vector of numbers`

In [13]:
# get the word count vector of the pattern string to encode the pattern string.

tv = TfidfVectorizer()
tv.fit(string_train)

x_train = tv.transform(string_train)
x_test = tv.transform(string_test)

x_train.shape, x_test.shape

((6549, 5596), (1638, 5596))

In [14]:
# save the CountVectorizer to disk

joblib.dump(tv, 'presence_TfidfVectorizer.joblib')

['presence_TfidfVectorizer.joblib']

---
# Rough Idea about the effect of different classifiers
---

In [15]:
# Four models are tested:
# -- Logistic Regression
# -- Linear Support Vector Machine
# -- Random Forest
# -- Bernoulli Naive Bayes

classifiers = [LogisticRegression(), LinearSVC(), RandomForestClassifier(), BernoulliNB()]

In [16]:
# Calculate the accuracies of different classifiers using default settings.

acc = []
pre = []
cm = []

for clf in classifiers:
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    acc.append(metrics.accuracy_score(y_test, y_pred))
    pre.append(metrics.precision_score(y_test, y_pred, pos_label=0))
    cm.append(metrics.confusion_matrix(y_test, y_pred))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# List the accuracies of different classifiers.

for i in range(len(classifiers)):
    print("{} accuracy: {:.3f}".format(classifiers[i],acc[i]))
    print("{} precision: {:.3f}".format(classifiers[i],pre[i]))
    print("Confusion Matrix: {}".format(cm[i]))

LogisticRegression() accuracy: 0.991
LogisticRegression() precision: 1.000
Confusion Matrix: [[  24   15]
 [   0 1599]]
LinearSVC() accuracy: 0.995
LinearSVC() precision: 1.000
Confusion Matrix: [[  30    9]
 [   0 1599]]
RandomForestClassifier() accuracy: 0.994
RandomForestClassifier() precision: 1.000
Confusion Matrix: [[  29   10]
 [   0 1599]]
BernoulliNB() accuracy: 0.976
BernoulliNB() precision: 0.000
Confusion Matrix: [[   0   39]
 [   0 1599]]


---
# Bernoulli Naive Bayes Classifier


---
### `Use default setting of classifier hyperparameters`

In [18]:
clf_bnb = BernoulliNB().fit(x_train, y_train)

y_pred = clf_bnb.predict(x_test)

In [19]:
clf_bnb.get_params()

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}

---
`use the default setting of hyperparameters of the Bernoulli Naive Bayes classifier`

In [20]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9761904761904762
Precision: 0.0
Confusion Matrix:
 [[   0   39]
 [   0 1599]]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   1, 1638]])

---
### `Parameter Tunning of BernoulliNB classifier`
`Define the combination of parameters to be considered`

In [22]:
param_grid = {'alpha':[0,1], 
              'fit_prior':[True, False]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [23]:
gs = GridSearchCV(clf_bnb,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [24]:
best_bnb = gs.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    1.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.6s finished


In [25]:
scores_df = pd.DataFrame(best_bnb.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_alpha', 'param_fit_prior']]

,rank_test_score,mean_test_score,param_alpha,param_fit_prior
0,1,0.975874,1,True
1,2,0.975264,1,False
2,3,0.969920,0,True
3,4,0.961674,0,False


In [26]:
best_bnb.best_params_

{'alpha': 1, 'fit_prior': True}

In [27]:
y_pred_best = best_bnb.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   1 1638]]


In [28]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9761904761904762
Precision: 0.0
Confusion Matrix:
 [[   0   39]
 [   0 1599]]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---
`Save the best BernoulliNB model for future use`

In [29]:
# save the model to local disk

joblib.dump(best_bnb, 'bnb_presence_classifier.joblib')

['bnb_presence_classifier.joblib']

---
# Random Forest Classifier


---
### `Use default setting of classifier hyperparameters`

In [30]:
clf_rf = RandomForestClassifier().fit(x_train, y_train)

y_pred = clf_rf.predict(x_test)

In [31]:
clf_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [32]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9951159951159951
Precision: 1.0
Confusion Matrix:
 [[  31    8]
 [   0 1599]]


In [33]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,   31],
       [   1, 1607]])

---
### `Parameter Tunning of Random Forest classifier`
`Define the combination of parameters to be considered`

In [34]:
param_grid = {'bootstrap':[True,False], 
              'criterion':['gini','entropy'],
              'max_depth':[10,20,30,40,50, None],
              'min_samples_leaf':[1,2,4],
              'min_samples_split':[2,5,10],
              'n_estimators':[100,200,300]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [35]:
gs = GridSearchCV(clf_rf,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [36]:
best_rf = gs.fit(x_train,y_train)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 26.3min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 36.4min
[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed: 38.5min finished


In [37]:
scores_df = pd.DataFrame(best_rf.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_bootstrap', 'param_criterion','param_max_depth','param_min_samples_leaf','param_min_samples_split','param_n_estimators']]

,rank_test_score,mean_test_score,param_bootstrap,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators
0,1,0.997404,False,gini,None,1,10,300
1,2,0.997099,False,gini,None,2,10,300
2,2,0.997099,False,entropy,None,1,10,200
3,2,0.997099,False,entropy,None,2,2,100
4,2,0.997099,False,gini,None,1,5,100
...,...,...,...,...,...,...,...,...
643,644,0.976485,False,entropy,10,4,5,200
644,644,0.976485,True,entropy,10,2,5,300
645,644,0.976485,False,gini,10,4,10,300
646,644,0.976485,True,gini,10,4,5,200


In [38]:
best_rf.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 300}

In [39]:
y_pred_best = best_rf.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0   32]
 [   1 1606]]


In [40]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9957264957264957
Precision: 1.0
Confusion Matrix:
 [[  32    7]
 [   0 1599]]


---
`Save the best Random Forest model for future use`

In [41]:
# save the model to local disk

joblib.dump(best_rf, 'rf_presence_classifier.joblib')

['rf_presence_classifier.joblib']

---
# SVM Classifier


---
### `Use default setting of classifier hyperparameters`

In [42]:
clf_svm = LinearSVC().fit(x_train,y_train)

y_pred = clf_svm.predict(x_test)

In [43]:
clf_svm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [44]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9945054945054945
Precision: 1.0
Confusion Matrix:
 [[  30    9]
 [   0 1599]]


In [45]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,   30],
       [   1, 1608]])

---
### `Parameter Tunning of SVM classifier`
`Define the combination of parameters to be considered`

In [46]:
param_grid = {'C':[0.1,1,10,100],
              'penalty':['l1','l2']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [47]:
gs = GridSearchCV(clf_svm,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [48]:
best_svm = gs.fit(x_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.1s finished


In [49]:
scores_df = pd.DataFrame(best_svm.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_C']]

,rank_test_score,mean_test_score,param_penalty,param_C
0,1,0.996488,l2,10
1,2,0.996336,l2,1
2,3,0.995724,l2,100
3,4,0.994045,l2,0.1
4,5,NaN,l1,0.1
5,6,NaN,l1,1
6,7,NaN,l1,10
7,8,NaN,l1,100


In [50]:
best_svm.best_params_

{'C': 10, 'penalty': 'l2'}

In [51]:
y_pred_best = best_svm.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0   30]
 [   1 1608]]


In [52]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9932844932844933
Precision: 0.9666666666666667
Confusion Matrix:
 [[  29   10]
 [   1 1598]]


---
`Save the best SVM model for future use`

In [53]:
# save the model to local disk

joblib.dump(best_svm, 'svm_presence_classifier.joblib')

['svm_presence_classifier.joblib']

---
# Logistic Regression Classifier


---
### `Use default setting of classifier hyperparameters`

In [54]:
clf_lr = LogisticRegression().fit(x_train, y_train)

y_pred = clf_lr.predict(x_test)

In [55]:
clf_lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Logistic Regression classifier.`

In [56]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9908424908424909
Precision: 1.0
Confusion Matrix:
 [[  24   15]
 [   0 1599]]


In [57]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,   24],
       [   1, 1614]])

---
### `Parameter Tunning of Logistic Regression classifier`
`Define the combination of parameters to be considered`

In [58]:
param_grid = {'penalty':['l1','l2'], 
              'solver':['lbfgs','newton-cg','sag']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [59]:
gs = GridSearchCV(clf_lr,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [60]:
best_lr = gs.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished


In [61]:
scores_df = pd.DataFrame(best_lr.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_solver']]

,rank_test_score,mean_test_score,param_penalty,param_solver
0,1,0.990686,l2,lbfgs
1,1,0.990686,l2,newton-cg
2,1,0.990686,l2,sag
3,4,NaN,l1,lbfgs
4,5,NaN,l1,newton-cg
5,6,NaN,l1,sag


In [62]:
best_lr.best_params_

{'penalty': 'l2', 'solver': 'lbfgs'}

In [63]:
y_pred_best = best_lr.predict(x_test)

(unique, counts) = np.unique(y_pred_best, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0   24]
 [   1 1614]]


In [64]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_best))
print("Precision:", metrics.precision_score(y_test,y_pred_best, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred_best))

Accuracy: 0.9908424908424909
Precision: 1.0
Confusion Matrix:
 [[  24   15]
 [   0 1599]]


---
`Save the best Logistic Regression model for future use`

In [65]:
# save the model to local disk

joblib.dump(best_lr, 'lr_presence_classifier.joblib')

['lr_presence_classifier.joblib']